In [1]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
from src.component.data_info import*

In [2]:
#read the data
demand_data=pd.read_parquet((TRANSFORMED_DATA_DIR / 'tabular_dataNY.parquet'))
demand_data

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,date,sub_region_code,temperature_2m,target_demand_next_hour
0,1487.0,1422.0,1373.0,1336.0,1317.0,1307.0,1315.0,1343.0,1373.0,1395.0,...,1903.0,1933.0,1977.0,1994.0,1994.0,1958.0,2022-01-28 02:00:00+00:00,0,4.4085,1908.0
1,1624.0,1571.0,1497.0,1446.0,1418.0,1406.0,1395.0,1396.0,1422.0,1469.0,...,1920.0,1899.0,1886.0,1891.0,1942.0,1976.0,2022-01-29 00:00:00+00:00,0,2.6585,1976.0
2,1868.0,1836.0,1786.0,1715.0,1637.0,1583.0,1559.0,1552.0,1559.0,1588.0,...,1798.0,1773.0,1745.0,1732.0,1723.0,1764.0,2022-01-29 23:00:00+00:00,0,5.7085,1858.0
3,2005.0,2005.0,1989.0,1991.0,1922.0,1842.0,1757.0,1702.0,1661.0,1633.0,...,1687.0,1697.0,1690.0,1670.0,1653.0,1653.0,2022-01-30 21:00:00+00:00,0,2.4585,1662.0
4,1877.0,1981.0,1971.0,1971.0,1938.0,1883.0,1832.0,1739.0,1650.0,1599.0,...,1884.0,1904.0,1877.0,1856.0,1829.0,1795.0,2022-01-31 19:00:00+00:00,0,-1.4415,1770.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12788,2048.0,1970.0,2007.0,2143.0,2295.0,2502.0,2667.0,2666.0,2666.0,2625.0,...,1887.0,1835.0,1825.0,1858.0,1927.0,1999.0,2024-12-28 14:00:00+00:00,10,-1.0915,2091.0
12789,2169.0,2066.0,1994.0,1986.0,2034.0,2170.0,2382.0,2608.0,2758.0,2762.0,...,1834.0,1734.0,1666.0,1621.0,1617.0,1642.0,2024-12-29 12:00:00+00:00,10,-0.5915,1699.0
12790,2353.0,2330.0,2283.0,2228.0,2158.0,2061.0,2149.0,2240.0,2351.0,2566.0,...,2030.0,1886.0,1767.0,1656.0,1599.0,1568.0,2024-12-30 10:00:00+00:00,10,3.0085,1587.0
12791,2084.0,2293.0,2396.0,2313.0,2155.0,2044.0,2042.0,2123.0,2247.0,2333.0,...,2358.0,2252.0,2115.0,1963.0,1838.0,1740.0,2024-12-31 08:00:00+00:00,10,1.9085,1682.0


In [3]:
#split the data
from datetime import datetime, timezone
from src.component.data_info import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    demand_data,
    cutoff_date=datetime(2024, 9, 1, 0, 0, 0,tzinfo=timezone.utc),
    target_column_name='target_demand_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(11330, 675)
y_train.shape=(11330,)
X_test.shape=(1463, 675)
y_test.shape=(1463,)


In [4]:
# use only past demand data
# past_demand_columns = [c for c in X_train.columns if c.startswith('demand_')]
# X_train_only_numeric = X_train[past_demand_columns]
X_train_numurical=X_train.drop(['date'], axis=1)

## Baseline model (Linear regression)

In [5]:
#basleline model linear regrisson
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_numurical, y_train)

LinearRegression()

In [6]:
X_test_numeric = X_test.drop(['date'], axis=1)
y_pred = model.predict(X_test_numeric)
y_pred

array([1919.43746977, 1531.31767564, 1625.22766093, ..., 1593.34259147,
       1675.79645131, 1868.10183523])

In [7]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
def evaluate_model(y_test, y_pred):
    test_mae = mean_absolute_error(y_test, y_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_pred)
    return f"MAE is {test_mae:.4f} and MAPE is: {test_mape:.4f}"
    

In [8]:
evaluate_model(y_test, y_pred)

'MAE is 17.0237 and MAPE is: 0.0148'

## XGboost model

In [9]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
model_xgb.fit(X_train_numurical, y_train)
print(model_xgb)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)


In [10]:

y_pred_XGB = model_xgb.predict(X_test_numeric)
y_pred_XGB

array([1879.7296, 1562.837 , 1652.9828, ..., 1600.5839, 1686.099 ,
       1840.366 ], dtype=float32)

In [11]:
evaluate_model(y_test, y_pred_XGB)

'MAE is 28.6750 and MAPE is: 0.0214'

## LightBM model

In [12]:
import lightgbm as lgb

In [13]:
model_lgb = lgb.LGBMRegressor()
model_lgb.fit(X_train_numurical, y_train)
print(model_lgb)

d:\poetry_virtualenvs\src-o1faXYQI-py3.12\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\poetry_virtualenvs\src-o1faXYQI-py3.12\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.235206 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171625
[LightGBM] [Info] Number of data points in the train set: 11330, number of used features: 674
[LightGBM] [Info] Start training from score 1569.062754
LGBMRegressor()


In [14]:
y_pred_lgb=model_lgb.predict(X_test_numeric)
y_pred_lgb

array([1892.03175322, 1534.42598006, 1642.69952617, ..., 1585.33673325,
       1674.19795385, 1863.62365081])

In [15]:
evaluate_model(y_test,y_pred_lgb)

'MAE is 26.3632 and MAPE is: 0.0207'

as we can see the lighbm perform better so we will choose it and do hyperprameter tunning

## lightbm with feature_engineering

In [16]:

def average_demand_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    """
    Adds one column with the average rides from
    - 7 days ago
    - 14 days ago
    - 21 days ago
    - 28 days ago
    """
    X['average_demand_last_4_weeks'] = 0.25*(
    X[f'demand_previous_{7*24}_hour'] + \
    X[f'demand_previous_{2*7*24}_hour'] + \
    X[f'demand_previous_{3*7*24}_hour'] + \
    X[f'demand_previous_{4*7*24}_hour']
    )
    return X

In [17]:
from sklearn.preprocessing import FunctionTransformer

add_feature_average_demand_last_4_weeks = FunctionTransformer(
    average_demand_last_4_weeks, validate=False)
     

In [18]:
add_feature_average_demand_last_4_weeks.fit_transform(X_train)

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,date,sub_region_code,temperature_2m,average_demand_last_4_weeks
0,1487.0,1422.0,1373.0,1336.0,1317.0,1307.0,1315.0,1343.0,1373.0,1395.0,...,1903.0,1933.0,1977.0,1994.0,1994.0,1958.0,2022-01-28 02:00:00+00:00,0,4.408500,1749.00
1,1624.0,1571.0,1497.0,1446.0,1418.0,1406.0,1395.0,1396.0,1422.0,1469.0,...,1920.0,1899.0,1886.0,1891.0,1942.0,1976.0,2022-01-29 00:00:00+00:00,0,2.658500,1699.25
2,1868.0,1836.0,1786.0,1715.0,1637.0,1583.0,1559.0,1552.0,1559.0,1588.0,...,1798.0,1773.0,1745.0,1732.0,1723.0,1764.0,2022-01-29 23:00:00+00:00,0,5.708500,1722.50
3,2005.0,2005.0,1989.0,1991.0,1922.0,1842.0,1757.0,1702.0,1661.0,1633.0,...,1687.0,1697.0,1690.0,1670.0,1653.0,1653.0,2022-01-30 21:00:00+00:00,0,2.458500,1818.25
4,1877.0,1981.0,1971.0,1971.0,1938.0,1883.0,1832.0,1739.0,1650.0,1599.0,...,1884.0,1904.0,1877.0,1856.0,1829.0,1795.0,2022-01-31 19:00:00+00:00,0,-1.441500,1851.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11325,2804.0,2652.0,2552.0,2518.0,2556.0,2645.0,2864.0,3136.0,3393.0,3659.0,...,3935.0,3830.0,3638.0,3638.0,3492.0,3271.0,2024-08-28 03:00:00+00:00,10,16.358500,2602.75
11326,3475.0,3207.0,2993.0,2839.0,2743.0,2713.0,2746.0,2844.0,3076.0,3399.0,...,4503.0,4743.0,4845.0,4741.0,4540.0,4540.0,2024-08-29 01:00:00+00:00,10,22.758501,2701.50
11327,3987.0,3735.0,3478.0,3248.0,3054.0,2894.0,2780.0,2720.0,2712.0,2721.0,...,3208.0,3295.0,3332.0,3346.0,3316.0,3247.0,2024-08-30 00:00:00+00:00,10,24.658501,3168.75
11328,4016.0,3763.0,3575.0,3366.0,3136.0,2930.0,2771.0,2642.0,2555.0,2497.0,...,2559.0,2616.0,2690.0,2780.0,2925.0,3074.0,2024-08-30 22:00:00+00:00,10,17.108500,3096.00


In [19]:

from sklearn.base import BaseEstimator, TransformerMixin
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
class TemporalFeaturesEngineer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        X_ = X.copy()
        X_['date']= pd.to_datetime(X_['date'], format='%Y-%m-%d %H:%M:%S')
        
        # Generate numeric columns from datetime
        X_["hour"] = X_['date'].dt.hour
        X_["day_of_week"] = X_['date'].dt.dayofweek
        X_['month'] = X_['date'].dt.month
        X_['is_weekend'] = X_['day_of_week'].isin([5, 6]).astype(int)

        holidays = calendar().holidays(start=X_['date'].min(), end=X_['date'].max())
        X_['is_holiday'] = X_['date'].isin(holidays).astype(int)       
        
        return X_.drop(columns=['date'])

In [20]:
add_temporal_features = TemporalFeaturesEngineer()
add_temporal_features.fit_transform(X_train)

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_2_hour,demand_previous_1_hour,sub_region_code,temperature_2m,average_demand_last_4_weeks,hour,day_of_week,month,is_weekend,is_holiday
0,1487.0,1422.0,1373.0,1336.0,1317.0,1307.0,1315.0,1343.0,1373.0,1395.0,...,1994.0,1958.0,0,4.408500,1749.00,2,4,1,0,0
1,1624.0,1571.0,1497.0,1446.0,1418.0,1406.0,1395.0,1396.0,1422.0,1469.0,...,1942.0,1976.0,0,2.658500,1699.25,0,5,1,1,0
2,1868.0,1836.0,1786.0,1715.0,1637.0,1583.0,1559.0,1552.0,1559.0,1588.0,...,1723.0,1764.0,0,5.708500,1722.50,23,5,1,1,0
3,2005.0,2005.0,1989.0,1991.0,1922.0,1842.0,1757.0,1702.0,1661.0,1633.0,...,1653.0,1653.0,0,2.458500,1818.25,21,6,1,1,0
4,1877.0,1981.0,1971.0,1971.0,1938.0,1883.0,1832.0,1739.0,1650.0,1599.0,...,1829.0,1795.0,0,-1.441500,1851.00,19,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11325,2804.0,2652.0,2552.0,2518.0,2556.0,2645.0,2864.0,3136.0,3393.0,3659.0,...,3492.0,3271.0,10,16.358500,2602.75,3,2,8,0,0
11326,3475.0,3207.0,2993.0,2839.0,2743.0,2713.0,2746.0,2844.0,3076.0,3399.0,...,4540.0,4540.0,10,22.758501,2701.50,1,3,8,0,0
11327,3987.0,3735.0,3478.0,3248.0,3054.0,2894.0,2780.0,2720.0,2712.0,2721.0,...,3316.0,3247.0,10,24.658501,3168.75,0,4,8,0,0
11328,4016.0,3763.0,3575.0,3366.0,3136.0,2930.0,2771.0,2642.0,2555.0,2497.0,...,2925.0,3074.0,10,17.108500,3096.00,22,4,8,0,0


In [21]:

from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_demand_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train);

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.087359 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171928
[LightGBM] [Info] Number of data points in the train set: 11330, number of used features: 680
[LightGBM] [Info] Start training from score 1569.062754


In [22]:
import warnings
warnings.filterwarnings("ignore")

predictions = pipeline.predict(X_test)
print(' ')

In [23]:

from sklearn.metrics import mean_absolute_error
evaluate_model(y_test,predictions)

'MAE is 25.4574 and MAPE is: 0.0197'

In [24]:
from src.component.model_info import *

## lightBm with hyperprameter tunning

In [33]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

# from src.model_info import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100), 
        #"learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)
  
    }
       
    tss = KFold(n_splits=6)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [34]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=4)

[I 2025-02-01 16:04:59,324] A new study created in memory with name: no-name-d4e876d4-430a-4ce4-b26c-36c1b4d7d72b
[I 2025-02-01 16:06:49,261] Trial 0 finished with value: 66.61125448498285 and parameters: {'num_leaves': 189, 'feature_fraction': 0.3793371340437163, 'bagging_fraction': 0.3725027968646756, 'min_child_samples': 18}. Best is trial 0 with value: 66.61125448498285.
[I 2025-02-01 16:08:07,056] Trial 1 finished with value: 65.95961345185356 and parameters: {'num_leaves': 255, 'feature_fraction': 0.38425739250066127, 'bagging_fraction': 0.49068105431447845, 'min_child_samples': 50}. Best is trial 1 with value: 65.95961345185356.
[I 2025-02-01 16:08:47,569] Trial 2 finished with value: 81.58690371427969 and parameters: {'num_leaves': 218, 'feature_fraction': 0.3736059857671459, 'bagging_fraction': 0.26083046822777833, 'min_child_samples': 97}. Best is trial 1 with value: 65.95961345185356.
[I 2025-02-01 16:10:53,563] Trial 3 finished with value: 69.48403965673845 and parameters: 

In [35]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 255, 'feature_fraction': 0.38425739250066127, 'bagging_fraction': 0.49068105431447845, 'min_child_samples': 50}


In [36]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_demand_last_4_weeks at 0x0000022ACE8A4E00>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.49068105431447845,
                               feature_fraction=0.38425739250066127,
                               min_child_samples=50, num_leaves=255))])

In [37]:
predictions = pipeline.predict(X_test)
evaluate_model(y_test, predictions)

'MAE is 34.8712 and MAPE is: 0.0259'

In [33]:
# #save the model 
# import joblib
# from src.paths import MODELS_DIR

# # Save the pipeline to a file
# joblib.dump(pipeline, MODELS_DIR/'pipeline_model.pkl')


['D:\\Electricity_demand_predictor2\\models\\pipeline_model.pkl']

In [39]:
#plot the result
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [40]:
#plot the result
from src.plot import plot_one_sample

plot_one_sample(
    example_id=7,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)